In [1]:
import sys
import os
sys.path.append('/home/581/ll6859/Documents/')
from aercode import *
import matplotlib 
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np
import xarray as xr
import glob as gb
import cartopy.crs as ccrs
import seaborn as sns
from datetime import datetime

import dask
import warnings
from dask.distributed import Client

In [2]:
plt.style.use('default')
warnings.filterwarnings('ignore')
dask.config.set(**{'array.slicing.split_large_chunks': True})
client = Client()
client

2022-06-23 12:35:00,826 - distributed.diskutils - INFO - Found stale lock file and directory '/home/581/ll6859/Documents/CAMMPCAN/dask-worker-space/worker-twqm4m8z', purging
2022-06-23 12:35:00,844 - distributed.diskutils - INFO - Found stale lock file and directory '/home/581/ll6859/Documents/CAMMPCAN/dask-worker-space/worker-s09d1a9d', purging
2022-06-23 12:35:00,855 - distributed.diskutils - INFO - Found stale lock file and directory '/home/581/ll6859/Documents/CAMMPCAN/dask-worker-space/worker-zz5e_xrh', purging
2022-06-23 12:35:00,868 - distributed.diskutils - INFO - Found stale lock file and directory '/home/581/ll6859/Documents/CAMMPCAN/dask-worker-space/worker-ati2hyef', purging


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 16,Total memory: 44.92 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:45703,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 44.92 GiB
Comm: tcp://127.0.0.1:42865,Total threads: 4
Dashboard: http://127.0.0.1:43095/status,Memory: 11.23 GiB
Nanny: tcp://127.0.0.1:33473,


In [3]:
# pull out location of ship
f_path = '/home/581/ll6859/Documents/CAMMPCAN/cammpcan_uw/'
voyages = sorted(gb.glob(f_path+'201819_Voyage*.csv'))
uw = pd.concat([pd.read_csv(v) for v in voyages ])
uw = uw.set_index(pd.to_datetime(uw['date_time_utc']))
uw = uw[['latitude','longitude']]
uw = uw.resample('1D', kind='Date').mean().ffill()
uw.columns = ['lat','lon']

In [4]:
uw

,lat,lon
date_time_utc,,
2018-10-25,-44.129021,146.961698
2018-10-26,-47.932618,145.005544
2018-10-27,-51.631842,142.880163
2018-10-28,-55.699378,140.277589
2018-10-29,-59.148387,134.745530
...,...,...
2019-03-20,-54.580760,158.941042
2019-03-21,-54.579320,158.941199
2019-03-22,-52.883732,157.223753


In [5]:
# extract chemistry data
m_path = '/g/data/jk72/slf563/ACCESS/output/cg893/daily/'
mod = 'cg893a'
dt = 'pche'
key = uw

chetrack = df_md(key, m_path, mod, dt)

In [6]:
# convert chemistry units
chetrack = chem_unit_conversions(chetrack)

In [7]:
# extract aerosol data
m_path = '/g/data/jk72/slf563/ACCESS/output/cg893/daily/tmp/'
mod = 'cg893a'
dt = 'paer'
key = uw

aertrack = df_md(key, m_path, mod, dt)

In [8]:
# extract meteorology data
m_path = '/g/data/jk72/slf563/ACCESS/output/cg893/daily/'
mod = 'cg893a'
dt = 'pmet'
key = uw

mettrack = df_md(key, m_path, mod, dt)

In [9]:
# extract u component wind data
m_path = '/g/data/jk72/slf563/ACCESS/output/cg893/daily/'
mod = 'cg893a'
dt = 'pmetu'
key = uw

metutrack = df_md(key, m_path, mod, dt)

In [10]:
# extract v component wind data
m_path = '/g/data/jk72/slf563/ACCESS/output/cg893/daily/'
mod = 'cg893a'
dt = 'pmetv'
key = uw

metvtrack = df_md(key, m_path, mod, dt)

In [11]:
# calculate air density from temperature
mettrack = calc_density2(mettrack)

In [12]:
# convert aerosol units
aertrack = aero_unit_conversions2(aertrack, mettrack)

In [13]:
# select data at 20m
aertrack = aertrack.isel(z3_hybrid_height=0, z0_hybrid_height=0).expand_dims('z0_hybrid_height').transpose()

In [14]:
# calculate aerosol number concentrations
aertrack = nt_calcs(aertrack)

In [15]:
# calculate CCN number concentrations
aertrack = ccn_calcs(aertrack)

In [16]:
# long name for aerosol mode and composition fields
aertrack['field431'] = aertrack['field431'].assign_attrs({'long_name':'Dust division 1'})
aertrack['field432'] = aertrack['field432'].assign_attrs({'long_name':'Dust division 2'})
aertrack['field433'] = aertrack['field433'].assign_attrs({'long_name':'Dust division 3'})
aertrack['field434'] = aertrack['field434'].assign_attrs({'long_name':'Dust division 4'})
aertrack['field435'] = aertrack['field435'].assign_attrs({'long_name':'Dust division 5'})
aertrack['field436'] = aertrack['field436'].assign_attrs({'long_name':'Dust division 6'})

aertrack['field34101'] = aertrack['field34101'].assign_attrs({'long_name':'NUCLEATION MODE (SOLUBLE) NUMBER'})
aertrack['field34102'] = aertrack['field34102'].assign_attrs({'long_name':'NUCLEATION MODE (SOLUBLE) H2SO4'})
aertrack['field34103'] = aertrack['field34103'].assign_attrs({'long_name':'AITKEN MODE (SOLUBLE) NUMBER'})
aertrack['field34104'] = aertrack['field34104'].assign_attrs({'long_name':'AITKEN MODE (SOLUBLE) H2SO4'})
aertrack['field34105'] = aertrack['field34105'].assign_attrs({'long_name':'AITKEN MODE (SOLUBLE) BC'})
aertrack['field34106'] = aertrack['field34106'].assign_attrs({'long_name':'AITKEN MODE (SOLUBLE) OM'})
aertrack['field34107'] = aertrack['field34107'].assign_attrs({'long_name':'ACCUMULATION MODE (SOLUBLE) NUMBER'})
aertrack['field34108'] = aertrack['field34108'].assign_attrs({'long_name':'ACCUMULATION MODE (SOL) H2SO4'})
aertrack['field34109'] = aertrack['field34109'].assign_attrs({'long_name':'ACCUMULATION MODE (SOL) BC'})
aertrack['field34110'] = aertrack['field34110'].assign_attrs({'long_name':'ACCUMULATION MODE (SOL) OM'})
aertrack['field34111'] = aertrack['field34111'].assign_attrs({'long_name':'ACCUMULATION MODE (SOL) SEA SALT'})
aertrack['field34113'] = aertrack['field34113'].assign_attrs({'long_name':'COARSE MODE (SOLUBLE) NUMBER'})
aertrack['field34114'] = aertrack['field34114'].assign_attrs({'long_name':'COARSE MODE (SOLUBLE) H2SO4'})
aertrack['field34115'] = aertrack['field34115'].assign_attrs({'long_name':'COARSE MODE (SOLUBLE) BC'})
aertrack['field34116'] = aertrack['field34116'].assign_attrs({'long_name':'COARSE MODE (SOLUBLE) OM'})
aertrack['field34117'] = aertrack['field34117'].assign_attrs({'long_name':'COARSE MODE (SOLUBLE) SEA SALT'})
aertrack['field34119'] = aertrack['field34119'].assign_attrs({'long_name':'AITKEN MODE (INSOLUBLE) NUMBER'})
aertrack['field34120'] = aertrack['field34120'].assign_attrs({'long_name':'AITKEN MODE (INSOLUBLE) BC'})
aertrack['field34121'] = aertrack['field34121'].assign_attrs({'long_name':'AITKEN MODE (INSOLUBLE) OM'})
aertrack['field34126'] = aertrack['field34126'].assign_attrs({'long_name':'NUCLEATION MODE (SOLUBLE) OM'})

In [17]:
# long name and unit changes for calculated aerosol fields
aertrack['N10'] = aertrack['N10'].assign_attrs({'units':'cm-3'})
aertrack['N10'] = aertrack['N10'].assign_attrs({'long_name':'Condensation Nuclei (>5r)'}) 

aertrack['N3'] = aertrack['N3'].assign_attrs({'units':'cm-3'})
aertrack['N3'] = aertrack['N3'].assign_attrs({'long_name':'Condensation Nuclei (>1.5r)'}) 

aertrack['CCN40'] = aertrack['CCN40'].assign_attrs({'units':'cm-3'})
aertrack['CCN40'] = aertrack['CCN40'].assign_attrs({'long_name':'Cloud Condensation Nuclei (>20r)'}) 

aertrack['CCN50'] = aertrack['CCN50'].assign_attrs({'units':'cm-3'})
aertrack['CCN50'] = aertrack['CCN50'].assign_attrs({'long_name':'Cloud Condensation Nuclei (>25r)'}) 

aertrack['CCN60'] = aertrack['CCN60'].assign_attrs({'units':'cm-3'})
aertrack['CCN60'] = aertrack['CCN60'].assign_attrs({'long_name':'Cloud Condensation Nuclei (>30r)'}) 

aertrack['DustSum'] = aertrack.field431 + aertrack.field432 + aertrack.field433 + aertrack.field434 + aertrack.field435 + aertrack.field436
aertrack['DustSum'] = aertrack['DustSum'].assign_attrs({'units':'mol/cm-3'})
aertrack['DustSum'] = aertrack['DustSum'].assign_attrs({'long_name':'Sum of dust divisions'}) 

In [18]:
# drop height coordinate from lat and lon
aertrack['lat'] = aertrack['lat'].drop('z0_hybrid_height').squeeze()
aertrack['lon'] = aertrack['lon'].drop('z0_hybrid_height').squeeze()

In [19]:
# calculate aerosol size distribution
aerdist = calc_size_dists(aertrack)

In [20]:
# merges aerosol, meteorology and chemistry data together
camptrack = xr.merge([aertrack,mettrack,metutrack,metvtrack,chetrack])

In [21]:
# drops calculated density field
camptrack = camptrack.drop('density')

In [22]:
# drops redundant height coordinates
camptrack = camptrack.drop('z3_hybrid_height')
camptrack = camptrack.drop('z1_hybrid_height').squeeze()

In [23]:
# adds wavelength attribute to the z_pseudo2 coordinate
camptrack['z_pseudo2'].attrs['wavelengths'] = '380 nm, 440 nm, 550 nm, 670 nm, 870 nm, 1020 nm'

In [24]:
# remove time component from datetime coordinate, invert datetime order
camptrack['time'] = camptrack.indexes['time'].normalize()
camptrack['time'] = camptrack['time'][::-1]
aerdist['Time'] = aerdist['Time'][::-1]

In [25]:
# add attributes to the time coordinate
camptrack['time'].attrs['standard_name'] ='time'
camptrack['time'].attrs['axis'] ='T'

aerdist['Time'].attrs['standard_name'] ='time'
aerdist['Time'].attrs['axis'] ='T'

In [26]:
# add metadata attributes to netCDF

camptrack.attrs['title'] = 'ACCESS-AM2 model output along the 2018-2019 CAMMPCAN Aurora Australis voyage tracks'
camptrack.attrs['contacts'] = 'Liam.Lamprey@utas.edu.au\nsonya.fiddes@utas.edu.au'
camptrack.attrs['description'] = 'Output from ACCESS-AM2 run for CMIP6 (research.csiro.au/access/cmip6/) with full chemistry and no ocean, nudged with ERA5, along the path of the Aurora Australis during the 2018-2019 CAMMPCAN project.'
camptrack.attrs['website'] = 'findanexpert.unimelb.edu.au/project/102792-cammpcan---chemical-and-mesoscale-mechanisms-of-polar-cell-aerosol-nucleation'

In [27]:
# add metadata attributes to netCDF

aerdist.attrs['title'] = 'ACCESS-AM2 model aerosol size distributions along the 2018-2019 CAMMPCAN Aurora Australis voyage tracks'
aerdist.attrs['contacts'] = 'Liam.Lamprey@utas.edu.au\nsonya.fiddes@utas.edu.au'
aerdist.attrs['description'] = 'Aerosol size distributions calculated using output from the ACCESS-AM2 run for CMIP6 (research.csiro.au/access/cmip6/) with full chemistry and no ocean, nudged with ERA5, along the path of the Aurora Australis during the 2018-2019 CAMMPCAN project.'
aerdist.attrs['website'] = 'findanexpert.unimelb.edu.au/project/102792-cammpcan---chemical-and-mesoscale-mechanisms-of-polar-cell-aerosol-nucleation'

In [28]:
# updates netCDF history attribute
camptrack = camptrack.assign_attrs(history='/scratch/jk72/slf563/cylc-run/u-cg893/share/data/History_Data/cg893a.paer20190325 converted to netCDF by slf563 on 2022-03-30. Merged aer, chem and met into netCDF by ll6859 on 2022-05-11.')

In [29]:
# adds netCDF history attribute
aerdist = aerdist.assign_attrs(history='Aerosol size distributions calculated with /g/data/q90/ll6859/aa1819_cg893_track.nc by ll6859 on 2022-05-11')

In [30]:
# print size of campaign data
camptrack.nbytes

19188676

In [31]:
# print size of aerosol distribution data
aerdist.nbytes

64424

In [32]:
# convert campaign data to netCDF
camptrack.load().to_netcdf(path='/g/data/q90/ll6859/aa1819_cg893_track.nc')

In [33]:
# convert aerosol size distribution data to netCDF
aerdist.load().to_netcdf(path='/g/data/q90/ll6859/aa1819_cg893_aerdist.nc')

In [34]:
# pull out variable names
list(camptrack.keys())

['field38237',
 'field38238',
 'field38239',
 'field38240',
 'field38241',
 'field38242',
 'field38243',
 'field38244',
 'field38245',
 'field38246',
 'field38247',
 'field38248',
 'field38249',
 'field38250',
 'field38251',
 'field38261',
 'field38262',
 'field38263',
 'field38264',
 'field38265',
 'field38266',
 'field38267',
 'field38268',
 'field38269',
 'field38270',
 'field38271',
 'field38272',
 'field38273',
 'field38274',
 'field38275',
 'field38284',
 'field38285',
 'field38286',
 'field38287',
 'field38288',
 'field38289',
 'field38290',
 'field38291',
 'field38292',
 'field38294',
 'field38295',
 'field38296',
 'field38297',
 'field38298',
 'field38301',
 'field38302',
 'field38303',
 'field38304',
 'field38305',
 'field38319',
 'field38320',
 'field38321',
 'field38322',
 'field38323',
 'field38326',
 'field38327',
 'field38328',
 'field38329',
 'field38338',
 'field38339',
 'field38342',
 'field38343',
 'field38346',
 'field38347',
 'field38354',
 'field38356',
 'field383

In [35]:
# pull out variable long names
for i in list(camptrack.keys()):
    if hasattr(camptrack[i], "long_name")==True:
        print(camptrack[i].long_name)
    else:
        print(np.nan)

NUC SCAVENGING H2SO4 NUCLN. (SOL)
NUC SCAVENGING H2SO4 AITKEN (SOL)
NUC SCAVENGING H2SO4 ACCUMULTN (SOL)
NUC SCAVENGING H2SO4 COARSE (SOL)
NUC SCAVENGING SEA-SALT ACCUM. (SOL)
NUC SCAVENGING SEA-SALT COARSE (SOL)
NUC SCAVENGING BLK CARB AITKEN (SOL)
NUC SCAVENGING BLK CARB ACCUM. (SOL)
NUC SCAVENGING BLK CARB COARSE (SOL)
NUC SCAVENGING BLK CARB AITKEN (INS)
NUC SCAVENGING OM NUCLEATION (SOL)
NUC SCAVENGING OM AITKEN (SOL)
NUC SCAVENGING OM ACCUMULATION (SOL)
NUC SCAVENGING OM COARSE (SOL)
NUC SCAVENGING OM AITKEN (INS)
IMP SCAVENGING H2SO4 NUCLN. (SOL)
IMP SCAVENGING H2SO4 AITKEN (SOL)
IMP SCAVENGING H2SO4 ACCUMULTN (SOL)
IMP SCAVENGING H2SO4 COARSE (SOL)
IMP SCAVENGING SEA-SALT ACCUM. (SOL)
IMP SCAVENGING SEA-SALT COARSE (SOL)
IMP SCAVENGING BLK CARB AITKEN (SOL)
IMP SCAVENGING BLK CARB ACCUM. (SOL)
IMP SCAVENGING BLK CARB COARSE (SOL)
IMP SCAVENGING BLK CARB AITKEN (INS)
IMP SCAVENGING OM NUCLEATION (SOL)
IMP SCAVENGING OM AITKEN (SOL)
IMP SCAVENGING OM ACCUMULATION (SOL)
IMP SCAVEN

In [36]:
# pull out variable coordinates
for i in list(camptrack.keys()):
    print(list(camptrack[i].coords))

['z0_hybrid_height', 'time']
['z0_hybrid_height', 'time']
['z0_hybrid_height', 'time']
['z0_hybrid_height', 'time']
['z0_hybrid_height', 'time']
['z0_hybrid_height', 'time']
['z0_hybrid_height', 'time']
['z0_hybrid_height', 'time']
['z0_hybrid_height', 'time']
['z0_hybrid_height', 'time']
['z0_hybrid_height', 'time']
['z0_hybrid_height', 'time']
['z0_hybrid_height', 'time']
['z0_hybrid_height', 'time']
['z0_hybrid_height', 'time']
['z0_hybrid_height', 'time']
['z0_hybrid_height', 'time']
['z0_hybrid_height', 'time']
['z0_hybrid_height', 'time']
['z0_hybrid_height', 'time']
['z0_hybrid_height', 'time']
['z0_hybrid_height', 'time']
['z0_hybrid_height', 'time']
['z0_hybrid_height', 'time']
['z0_hybrid_height', 'time']
['z0_hybrid_height', 'time']
['z0_hybrid_height', 'time']
['z0_hybrid_height', 'time']
['z0_hybrid_height', 'time']
['z0_hybrid_height', 'time']
['z0_hybrid_height', 'time']
['z0_hybrid_height', 'time']
['z0_hybrid_height', 'time']
['z0_hybrid_height', 'time']
['z0_hybrid_he

In [37]:
# pull out variable units
for i in list(camptrack.keys()):
    if hasattr(camptrack[i], "units")==True:
        print(camptrack[i].units)
    else:
        print(np.nan)

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
unspecified
unspecified
unspecified
unspecified
unspecified
nan
nan
nan
nan
nan
mol/cm-3
mol/cm-3
mol/cm-3
mol/cm-3
mol/cm-3
mol/cm-3
unspecified
unspecified
unspecified
unspecified
unspecified
unspecified
unspecified
nan
cm-3
mol/cm-3
cm-3
mol/cm-3
mol/cm-3
mol/cm-3
cm-3
mol/cm-3
mol/cm-3
mol/cm-3
mol/cm-3
cm-3
mol/cm-3
mol/cm-3
mol/cm-3
mol/cm-3
cm-3
mol/cm-3
mol/cm-3
mol/cm-3
nan
degrees_north
degrees_east
cm-3
cm-3
cm-3
cm-3
cm-3
mol/cm-3
1
kg kg-1
1
m s-1
kg kg-1
nan
Pa
Pa
W m-2
W m-2
W m-2
W m-2
W m-2
W m-2
W m-2
W m-2
1
W m-2
W m-2
W m-2
W m-2
nan
unspecified
unspecified
unspecified
unspecified
unspecified
unspecified
unspecified
unspecified
unspecified